In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [8]:
# Hyperparameters
batch_size = 64
learning_rate = 1e-3
num_epochs = 20

# Transform: Normalize and Flatten Images
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Lambda(lambda x: x.view(-1))  # Flatten the image
])

# Load CIFAR10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Files already downloaded and verified
Files already downloaded and verified
cuda


<a id="5"></a> <br>
### Convolutional Neural Network (CNN)

In [16]:
# 🔥 Channel Attention (Squeeze-and-Excitation Block)
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=1):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # Global pooling
        self.fc1 = nn.Linear(in_channels, in_channels // reduction)  # Reduce dim
        self.fc2 = nn.Linear(in_channels // reduction, in_channels)  # Restore dim
        self.sigmoid = nn.Sigmoid()  # Attention weights

    def forward(self, x):
        b, c, _, _ = x.size()
        out = self.global_avg_pool(x).view(b, c)  # Squeeze
        out = F.relu(self.fc1(out))  # FC layer 1
        out = self.sigmoid(self.fc2(out))  # FC layer 2 with Sigmoid
        out = out.view(b, c, 1, 1)  # Reshape
        return x * out  # Scale original feature maps

# 🔥 Spatial Attention
class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)  # 7x7 conv
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)  # Avg pooling across channels
        max_out, _ = torch.max(x, dim=1, keepdim=True)  # Max pooling across channels
        out = torch.cat([avg_out, max_out], dim=1)  # Concatenate channel-wise
        out = self.conv(out)  # Apply Conv
        return x * self.sigmoid(out)  # Apply attention weights


# Create CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolutional layers with BatchNorm
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm for 32 feature maps
        self.se1 = SEBlock(32)  # Channel Attention

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)  # BatchNorm for 64 feature maps
        self.se2 = SEBlock(64)  # Channel Attention

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # BatchNorm for 128 feature maps
        self.sa = SpatialAttention()  # Spatial Attention

        self.pool = nn.MaxPool2d(2, 2)  # Downsampling
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)  # BatchNorm for FC layer
        
        self.fc2 = nn.Linear(256, 10)  # Output layer for 10 classes

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Conv -> BatchNorm -> ReLU -> Pool
        x = self.se1(x)  # Apply Channel Attention

        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.se2(x)  # Apply Channel Attention

        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.sa(x)  # Apply Channel Attention

        x = torch.flatten(x, 1)  # Flatten for FC layer
        x = F.relu(self.bn_fc1(self.fc1(x)))  # FC -> BatchNorm -> ReLU
        x = self.fc2(x)  # No activation for final layer (handled by loss function)
        
        return x


model = CNNModel().to(device)
error = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [17]:
num_epochs = 25

# Traning the Model
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for (images, labels) in train_loader:
        images = images.to(device).view(-1, 3, 32, 32)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = error(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss += loss.item()
        
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Predict test dataset
    for images, labels in test_loader: 
        images = images.to(device).view(-1, 3, 32, 32)
        labels = labels.to(device)

        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[1]
        total += len(labels)
        correct += (predicted == labels).sum()
    
    accuracy = 100 * correct / float(total)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_loss:.4f}, Acc: {accuracy:.4f}%")

Epoch [0/25], Loss: 0.0011, Acc: 70.5500%
Epoch [1/25], Loss: 0.0009, Acc: 74.6200%
Epoch [2/25], Loss: 0.0005, Acc: 76.4900%
Epoch [3/25], Loss: 0.0014, Acc: 77.8500%
Epoch [4/25], Loss: 0.0003, Acc: 78.2300%
Epoch [5/25], Loss: 0.0005, Acc: 78.3300%
Epoch [6/25], Loss: 0.0005, Acc: 77.8700%
Epoch [7/25], Loss: 0.0002, Acc: 78.0600%
Epoch [8/25], Loss: 0.0003, Acc: 77.2400%
Epoch [9/25], Loss: 0.0010, Acc: 77.8100%
Epoch [10/25], Loss: 0.0004, Acc: 78.5900%
Epoch [11/25], Loss: 0.0007, Acc: 78.0100%
Epoch [12/25], Loss: 0.0002, Acc: 78.0800%
Epoch [13/25], Loss: 0.0000, Acc: 78.0100%
Epoch [14/25], Loss: 0.0001, Acc: 77.6400%
Epoch [15/25], Loss: 0.0001, Acc: 77.6900%
Epoch [16/25], Loss: 0.0001, Acc: 77.3000%
Epoch [17/25], Loss: 0.0001, Acc: 77.7500%
Epoch [18/25], Loss: 0.0003, Acc: 78.6100%
Epoch [19/25], Loss: 0.0000, Acc: 77.7700%
Epoch [20/25], Loss: 0.0001, Acc: 77.4900%
Epoch [21/25], Loss: 0.0010, Acc: 77.1500%
Epoch [22/25], Loss: 0.0000, Acc: 78.1800%
Epoch [23/25], Loss: 

In [11]:
# # visualization loss 
# plt.plot(iteration_list,loss_list)
# plt.xlabel("Number of iteration")
# plt.ylabel("Loss")
# plt.title("CNN: Loss vs Number of iteration")
# plt.show()

# # visualization accuracy 
# plt.plot(iteration_list,accuracy_list,color = "red")
# plt.xlabel("Number of iteration")
# plt.ylabel("Accuracy")
# plt.title("CNN: Accuracy vs Number of iteration")
# plt.show()